In [1]:
# load packages
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.impute import SimpleImputer

## Original Data Combination
We combined all records of each individual for both of training dataset A and testing dataset B, and removed hospital-level variables `Unit1`, `Unit2`, `HospAdmTime`, and `ICULOS` since the information in these columns is not related to individual-level prediction.

### training A
below can't be runned for future convenience, comment them but doesn't mean useless

In [2]:
# setpath = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_original'
# savepath = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_original_noUnit/'

In [3]:
# subjectid = sorted(os.listdir(setpath))
# for sid in subjectid:
#     df1 = pd.read_csv(setpath + '/' + sid, sep = "|")
#     df1 = df1.drop(['Unit1', 'Unit2', 'HospAdmTime','ICULOS'], axis=1)
#     df1['SubjectID'] = [sid[0:7]] * len(df1)
#     df1.to_csv(savepath + sid, index = False)

In [4]:
# with open("/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_original_noUnit.psv", "wb") as fout:
#     for i, csv in enumerate(subjectid):
#         if i == 0:
#             with open(savepath + csv, "rb") as f:
#                 fout.write(f.read())
#       # now the rest:    
#         with open(savepath  + csv, "rb") as f:
#             next(f) 
#             fout.write(f.read())

### testing B
below can't be runned for future convenience, comment them but doesn't mean useless

In [5]:
# setpathB = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_original'
# savepathB = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_original_noUnit/'

# vars = ['HR','O2Sat','Temp','SBP','MAP','DBP','Resp','EtCO2','BaseExcess','HCO3','FiO2','pH','PaCO2','SaO2',
#  'AST','BUN','Alkalinephos','Calcium','Chloride','Creatinine','Bilirubin_direct','Glucose','Lactate','Magnesium',
#  'Phosphate','Potassium','Bilirubin_total','TroponinI','Hct','Hgb','PTT','WBC','Fibrinogen','Platelets','Age',
#         'Gender']

In [6]:
# subjectidB = sorted(os.listdir(setpathB))
# for sid in subjectidB:
#     df2 = pd.read_csv(setpathB + '/' + sid, sep = "|")
#     df2 = df2.drop(['Unit1', 'Unit2', 'HospAdmTime','ICULOS'], axis=1)
#     df2['SubjectID'] = [sid[0:7]] * len(df2)
#     df2.to_csv(savepathB + sid, index = False)

In [7]:
# with open("/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_original_noUnit.psv", "wb") as fout:
#     for i, csv in enumerate(subjectidB):
#         if i == 0:
#             with open(savepathB + csv, "rb") as f:
#                 fout.write(f.read())
#       # now the rest:    
#         with open(savepathB  + csv, "rb") as f:
#             next(f) 
#             fout.write(f.read())

## Cleaning Training Dataset A and Testing Dataset B
First, we removed column of `ETCO2` was also dropped in both dataset A and dataset B because it only contains `NA` value.  

Then, we spent time in the imputation of missing values. We assumed that each patient has unique characteristic and every value has ongoing time specified. Thus, primarily, the `NAs` were imputed based on the last or next records of the missing values, meaning that if there is a value in previous or next time period, then impute with the previous or the next one. Otherwise, we replaced the mean for continuous variables and mode for categorical variables if the values are missing. 

Finally, considering the great benefit of early detection of Sepsis, we extracted the initial situation of patients, i.e. using the laboratory results and demographical features of the first time when patients visited hospitals. This gave us 20,336 records in training set and 20,000 data in test set. Notice that the final datasets are imbalanced: percentages of $0$ (do not develop sepsis) in the `SepsisLabel` in datasets A and B are around 91% and 94%, respectively. They will be balanced before modeling process later. 

In [8]:
A_original = pd.read_csv("dataset/training_setA/training_original_noUnit.psv",sep = ",")
A_original

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel,SubjectID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,0,p000001
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,0,p000001
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,0,p000001
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,0,p000001
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,0,p000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790264,88.0,98.0,NaN,135.0,81.00,64.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,1,p020643
790265,96.0,98.0,38.72,174.0,97.00,72.0,16.0,NaN,2.0,NaN,...,27.8,NaN,NaN,NaN,NaN,NaN,62.29,1,1,p020643
790266,140.0,97.0,NaN,133.0,81.50,62.5,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,1,p020643
790267,120.0,96.0,NaN,154.0,118.00,105.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,1,p020643


In [9]:
B_original = pd.read_csv("dataset/training_setB/testing_original_noUnit.psv",sep = ",")
B_original

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel,SubjectID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73,1,0,p100001
1,93.0,92.5,NaN,110.0,76.0,56.0,22.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73,1,0,p100001
2,91.0,96.0,NaN,108.0,84.5,72.0,23.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73,1,0,p100001
3,93.0,98.0,NaN,123.0,87.0,61.0,21.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73,1,0,p100001
4,93.0,95.0,NaN,110.0,81.0,70.0,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73,1,0,p100001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762014,80.0,96.0,NaN,115.0,87.0,65.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62,0,0,p120000
762015,74.0,97.0,NaN,114.0,83.0,67.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62,0,0,p120000
762016,78.0,98.0,NaN,110.0,83.0,69.0,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62,0,0,p120000
762017,82.0,99.0,36.6,124.0,91.0,71.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62,0,0,p120000


In [10]:
# missing values in A
A_original.isna().sum()

HR                   61194
O2Sat                95089
Temp                523358
SBP                 120213
MAP                  80870
DBP                 380351
Resp                 77262
EtCO2               790269
BaseExcess          707881
HCO3                726650
FiO2                678110
pH                  699647
PaCO2               720975
SaO2                751105
AST                 778448
BUN                 725791
Alkalinephos        778736
Calcium             750949
Chloride            724490
Creatinine          737780
Bilirubin_direct    789087
Glucose             693611
Lactate             763126
Magnesium           728786
Phosphate           750371
Potassium           704431
Bilirubin_total     780575
TroponinI           789304
Hct                 697209
Hgb                 720446
PTT                 751963
WBC                 730919
Fibrinogen          784239
Platelets           738768
Age                      0
Gender                   0
SepsisLabel              0
S

In [11]:
# missing values in B
B_original.isna().sum()

HR                   92212
O2Sat               107659
Temp                503689
SBP                 106066
MAP                 112414
DBP                 106259
Resp                161079
EtCO2               704383
BaseExcess          760255
HCO3                760608
FiO2                744809
pH                  745061
PaCO2               745006
SaO2                747618
AST                 748656
BUN                 719926
Alkalinephos        748610
Calcium             710005
Chloride            757330
Creatinine          719889
Bilirubin_direct    760211
Glucose             593154
Lactate             747716
Magnesium           725547
Phosphate           739614
Potassium           703328
Bilirubin_total     748571
TroponinI           748203
Hct                 717643
Hgb                 717248
PTT                 754626
WBC                 721919
Fibrinogen          757807
Platelets           721308
Age                      0
Gender                   0
SepsisLabel              0
S

In [12]:
## drop EtCO2 since all is missing
A_original = A_original.drop(columns = ['EtCO2'])
B_original = B_original.drop(columns = ['EtCO2'])

below can't be runned for future convenience, comment them but doesn't mean useless

In [13]:
## dataset A
# setpath = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_original'
# subjectid = sorted(os.listdir(setpath))
# vars = ['HR','O2Sat','Temp','SBP','MAP','DBP','Resp','EtCO2','BaseExcess','HCO3','FiO2','pH','PaCO2','SaO2',
#  'AST','BUN','Alkalinephos','Calcium','Chloride','Creatinine','Bilirubin_direct','Glucose','Lactate','Magnesium',
#  'Phosphate','Potassium','Bilirubin_total','TroponinI','Hct','Hgb','PTT','WBC','Fibrinogen','Platelets','Age','Gender']

In [14]:
#  for sid in subjectid:
#         df1 = pd.read_csv(setpath + '/' + sid, sep = "|")
#         df1 = df1.drop(['Unit1', 'Unit2', 'HospAdmTime','ICULOS'], axis=1)
#         if(len(df1[df1['SepsisLabel'] == 1]) > 0 ):
#             index_a = list(map(lambda i : i == 1, df1['SepsisLabel'])).index(True)
#             df2 = pd.DataFrame(df1.iloc[0:index_a + 1])
#             df = pd.DataFrame(df1.iloc[index_a ]).T
            
#             for vname in vars:
#                 if(df[vname].isna().sum() == 1):
#                     if(df2[vname].isna().sum() < len(df2[vname])):
#                         index_b = np.where(~np.isnan(df2[vname]))[0]
#                         index_b = index_b[-1]
#                         df[vname] = df[vname].replace(np.nan, df1[vname][index_b])
#                     else:
#                         if(df1[vname].isna().sum() < len(df1[vname])):
#                             index_b = np.where(~np.isnan(df1[vname]))[0]
#                             index_b = index_b[0]
#                             df[vname] = df[vname].replace(np.nan, df1[vname][index_b])
#         else:
#             df = pd.DataFrame(df1.iloc[0]).T
#             for vname in vars:
#                 if(df[vname].isna().sum() == 1 and (df1[vname].isna().sum() < len(df1[vname]))):
#                     index_b = np.where(~np.isnan(df1[vname]))[0]
#                     index_b = index_b[0]
#                     df[vname] = df[vname].replace(np.nan, df1[vname][index_b])

#         df['SubjectID'] = sid[0:7]
#         if(sid == subjectid[0]):
#             df.to_csv('/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_clean1.psv', mode='a', index=False, header=True)
#         else:
#             df.to_csv('/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setA/training_clean1.psv', mode='a', index=False, header=False)

In [15]:
## dataset B
# setpathB = '/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_original'
# subjectidB = sorted(os.listdir(setpathB))

In [16]:
# for sid in subjectidB:
#         df1 = pd.read_csv(setpathB + '/' + sid, sep = "|")
#         df1 = df1.drop(['Unit1', 'Unit2', 'HospAdmTime','ICULOS'], axis=1)
#         if(len(df1[df1['SepsisLabel'] == 1]) > 0 ):
#             index_a = list(map(lambda i : i == 1, df1['SepsisLabel'])).index(True)
#             df2 = pd.DataFrame(df1.iloc[0:index_a + 1])
#             df = pd.DataFrame(df1.iloc[index_a ]).T
#             for vname in vars:
#                 if(df[vname].isna().sum() == 1):
#                     if(df2[vname].isna().sum() < len(df2[vname])):
#                         index_b = np.where(~np.isnan(df2[vname]))[0]
#                         index_b = index_b[-1]
#                         df[vname] = df[vname].replace(np.nan, df1[vname][index_b])
#                     else:
#                         if(df1[vname].isna().sum() < len(df1[vname])):
#                             index_b = np.where(~np.isnan(df1[vname]))[0]
#                             index_b = index_b[0]
#                             df[vname] = df[vname].replace(np.nan, df1[vname][index_b])
#         else:
#             df = pd.DataFrame(df1.iloc[0]).T
#             for vname in vars:
#                 if(df[vname].isna().sum() == 1 and (df1[vname].isna().sum() < len(df1[vname]))):
#                     index_b = np.where(~np.isnan(df1[vname]))[0]
#                     index_b = index_b[0]
#                     df[vname] = df[vname].replace(np.nan, df1[vname][index_b])

#         df['SubjectID'] = sid[0:7]
#         if(sid == subjectidB[0]):
#             df.to_csv('/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_clean1.psv', mode='a', index=False, header=True)
#         else:
#             df.to_csv('/Users/xiaoy/OneDrive/Desktop/biosta707_finalproject/training_setB/testing_clean1.psv', mode='a', index=False, header=False)

In [17]:
df_preA = pd.read_csv("dataset/training_setA/training_clean1.psv",sep = ",")
df_preA.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel,SubjectID
0,97.0,95.0,36.11,98.0,75.33,NaN,19.0,NaN,24.0,45.0,...,37.2,12.5,NaN,5.7,NaN,317.0,83.14,0.0,0.0,p000001
1,61.0,99.0,36.44,124.0,65.00,43.0,17.5,NaN,NaN,22.0,...,27.8,9.7,NaN,11.0,NaN,158.0,75.91,0.0,0.0,p000002
2,87.0,96.0,37.11,131.0,70.33,50.0,29.0,NaN,8.0,32.0,...,26.9,9.1,30.5,10.0,NaN,488.0,45.82,0.0,0.0,p000003
3,103.5,97.0,36.78,107.5,70.50,52.0,18.0,NaN,0.0,22.0,...,27.6,8.3,21.3,7.6,NaN,220.0,65.71,0.0,0.0,p000004
4,84.0,97.5,37.28,140.5,94.50,NaN,17.5,NaN,NaN,24.0,...,41.0,14.2,29.0,8.1,NaN,273.0,28.09,1.0,0.0,p000005


In [18]:
df_preB = pd.read_csv("dataset/training_setB/testing_clean1.psv",sep = ",")
df_preB.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel,SubjectID
0,93.0,92.5,36.5,110.0,76.0,56.0,22.0,NaN,NaN,NaN,...,35.3,11.3,NaN,10.8,NaN,170.0,73.0,1.0,0.0,p100001
1,56.5,100.0,35.2,129.0,75.0,52.5,23.0,35.0,NaN,NaN,...,31.4,11.1,NaN,13.2,NaN,85.0,83.0,1.0,0.0,p100002
2,130.0,96.0,36.6,129.0,87.0,72.0,24.0,NaN,NaN,NaN,...,32.3,10.8,NaN,15.5,NaN,337.0,48.0,1.0,0.0,p100003
3,65.0,100.0,36.7,140.0,84.0,65.0,14.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67.0,1.0,0.0,p100004
4,55.0,100.0,36.1,131.0,68.0,50.5,24.0,NaN,NaN,NaN,...,24.7,8.0,32.3,10.6,NaN,421.0,50.0,1.0,0.0,p100005


#### imputing NA values

In [19]:
## drop EtCO2 since all is missing
df_preA = df_preA.drop(columns = ['EtCO2'])
df_preB = df_preB.drop(columns = ['EtCO2'])

In [20]:
## Impute rest of Missing Value with mean (continuous) and mode (categorical)
df_preA_noID = df_preA.drop(columns = ['SubjectID'])
miss_mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
miss_mean_imputer = miss_mean_imputer.fit(df_preA_noID)
imputed_df_preA = miss_mean_imputer.transform(df_preA_noID)
imputed_df_preA = pd.DataFrame(data = imputed_df_preA, columns = df_preA_noID.columns)
df_finalA = pd.concat([df_preA['SubjectID'], imputed_df_preA], axis = 1)
df_finalA.head()

,SubjectID,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,...,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel
0,p000001,97.0,95.0,36.11,98.0,75.33,62.215276,19.0,24.000000,45.0,...,9.237423,37.2,12.5,37.300321,5.7,305.387222,317.0,83.14,0.0,0.0
1,p000002,61.0,99.0,36.44,124.0,65.00,43.000000,17.5,-0.462911,22.0,...,9.237423,27.8,9.7,37.300321,11.0,305.387222,158.0,75.91,0.0,0.0
2,p000003,87.0,96.0,37.11,131.0,70.33,50.000000,29.0,8.000000,32.0,...,9.237423,26.9,9.1,30.500000,10.0,305.387222,488.0,45.82,0.0,0.0
3,p000004,103.5,97.0,36.78,107.5,70.50,52.000000,18.0,0.000000,22.0,...,9.237423,27.6,8.3,21.300000,7.6,305.387222,220.0,65.71,0.0,0.0
4,p000005,84.0,97.5,37.28,140.5,94.50,62.215276,17.5,-0.462911,24.0,...,9.237423,41.0,14.2,29.000000,8.1,305.387222,273.0,28.09,1.0,0.0


In [21]:
## the percentage of label as 0 in training A
len(df_finalA[df_finalA['SepsisLabel'] == 0])/len(df_finalA)

0.911978756884343

In [22]:
## Impute Missing Value with mean (continuous) and mode (categorical)
df_preB_noID = df_preB.drop(columns = ['SubjectID'])
miss_mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
miss_mean_imputer = miss_mean_imputer.fit(df_preB_noID)
imputed_df_preB = miss_mean_imputer.transform(df_preB_noID)
imputed_df_preB = pd.DataFrame(data = imputed_df_preB, columns = df_preB_noID.columns)
df_finalB = pd.concat([df_preB['SubjectID'], imputed_df_preB], axis = 1)
df_finalB.head()

,SubjectID,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,...,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel
0,p100001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,-1.895609,23.948197,...,5.059726,35.300000,11.300000,40.478854,10.800000,289.458162,170.00000,73.0,1.0,0.0
1,p100002,56.5,100.0,35.2,129.0,75.0,52.5,23.0,-1.895609,23.948197,...,3.700000,31.400000,11.100000,40.478854,13.200000,289.458162,85.00000,83.0,1.0,0.0
2,p100003,130.0,96.0,36.6,129.0,87.0,72.0,24.0,-1.895609,23.948197,...,0.720000,32.300000,10.800000,40.478854,15.500000,289.458162,337.00000,48.0,1.0,0.0
3,p100004,65.0,100.0,36.7,140.0,84.0,65.0,14.0,-1.895609,23.948197,...,5.059726,32.848838,10.789035,40.478854,10.431536,289.458162,204.82405,67.0,1.0,0.0
4,p100005,55.0,100.0,36.1,131.0,68.0,50.5,24.0,-1.895609,23.948197,...,0.030000,24.700000,8.000000,32.300000,10.600000,289.458162,421.00000,50.0,1.0,0.0


In [23]:
## the percentage of label as 0
len(df_finalB[df_finalB['SepsisLabel'] == 0])/len(df_finalB)

0.9429

save fianl dataset A and B to csv

In [25]:
df_finalA.to_csv('dataset/training_setA/trainingA_final.csv', header=True, index=False)  
df_finalB.to_csv('dataset/training_setB/testingB_final.csv', header=True, index=False)

#### Right now, we are getting a more reasonal final dataset for future analysis, then rest of project will be run in R